In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
import warnings
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics as mt
import statsmodels.api as sm
from time import time
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ConfusionMatrix

In [ ]:
ccdefault=pd.read_csv('../data//UCI_Credit_Card.csv')
ccdefault.rename(columns={'default.payment.next.month':'DEFAULT'},inplace=True)
ccdefault.rename(columns={'PAY_0':'PAY_1'},inplace=True)

In [ ]:
ccdefault.info()

In [ ]:
mcc = ccdefault.copy()
# Y Response variable dataframe
mcc_Y = mcc['DEFAULT']
# Features with no predictive features with respect to resposne variable
mcredit = mcc.drop(['DEFAULT'],axis=1)
# Create inci_X Explanatory Variables DF to support the individual models
mcc_X = mcredit
mcc_X_rf = mcredit.copy()

In [ ]:
cc = ccdefault.copy() # taking a copy in memory
if 'DEFAULT' in cc:
    y = cc['DEFAULT'].values
    del cc['DEFAULT']
    del cc['ID']
    del cc['BILL_AMT1']
    del cc['BILL_AMT2']
    del cc['BILL_AMT3']
    del cc['BILL_AMT4']
    del cc['BILL_AMT5']
    del cc['BILL_AMT6']
    X = cc.values
num_cv_iterations = 10
num_instances = len(y)
cv_object = StratifiedShuffleSplit(n_splits = num_cv_iterations,
test_size = 0.20, train_size = 0.80, random_state=11)
cv_object.get_n_splits(X, y)
print(cv_object)
for train_index, test_index in cv_object.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
scl_obj = StandardScaler()
scl_obj.fit(X_train) # find scalings for each column that make this zero mean and unit std
# the line of code above only looks at training data to get mean and std and we can use it 
# to transform new feature data
X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) # apply those means and std to the test set (without snooping at the test set values)

## Modeling and Evaluation 3 
We planned to use 3 models for our classification. Random forest , Decision tree and KNN. 

### Random Forest model 

Random forest is an ensempble technique that uses set of decision trees to handle classification problem. It uses random vectors to choose features absed on which it creates decision trees. Classification is done with each of these decision trees and then a majority vote is taken to decide final class. 
Following images gives an overview  of decision tree algo. 

In [ ]:
from IPython.display import Image
Image(url ='../images/random_forest_tan.png')

In [ ]:
iter_num=0
times_rec=[]
decision_tree_classifiers = dict()
random_forest_classifiers = dict()
max_acc_rf= 0
trn_idx_max_acc_rf = []
tst_idx_max_acc_rf = []

#for loop 
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices] 
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
    random_forest = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

    random_forest.fit(X_train,y_train)  # train object
    y_hat = random_forest.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    if acc > max_acc_rf :
        trn_idx_max_acc_rf = train_indices
        tst_idx_max_acc_rf = test_indices
        max_acc_rf = acc
    
    print ("classification report for iteration {} ".format(iter_num))

    rf_cl_reporter = ClassificationReport(random_forest, classes=['non_default' , 'default'], support=True)

    rf_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
    rf_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
    rf_cl_reporter.show()   
    
    pr_curve = PrecisionRecallCurve(random_forest)
    pr_curve.fit(X_train, y_train)
    pr_curve.score(X_test, y_test)
    pr_curve.show()
    
    
    roc_curve = ROCAUC(random_forest, classes=["not_default", "default"])
    roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
    roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
    roc_curve.show() 
    

    cm = ConfusionMatrix(random_forest, classes=['not default' , 'default'])
    # Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
    cm.fit(X_train, y_train)
    # To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
    # and then creates the confusion_matrix from scikit-learn.
    cm.score(X_test, y_test)
    # How did we do?
    cm.show()
     
    iter_num+=1
    

#### Random forest with class_weight = balanced 
In above section we explored random forest with default parameters. Default parameters has 100 estimators and max_depth value 2 also used default class weight wich gives equal weight to each of the classes. However in the cases where claases are unbalanced we need to adjust weight of the classes so that they are properly represented. for example in our case we have two classes default and non default and each class will get equal weight. However 

In [ ]:
t0 = time()
random_forest = RandomForestClassifier(n_estimators=100, max_depth=2,
                         random_state=0, class_weight = 'balanced')

random_forest.fit(X_train,y_train)  # train object
y_hat = random_forest.predict(X_test) # get test set precitions

t1=time()
diff=np.array([t1-t0])

print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
times_rec=np.append(times_rec,diff)


# now let's get the accuracy and confusion matrix for this iterations of training/testing
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print ("classification report for iteration {} ".format(iter_num))

rf_cl_reporter = ClassificationReport(random_forest, classes=['non_default' , 'default'], support=True)

rf_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
rf_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
rf_cl_reporter.show()   

pr_curve = PrecisionRecallCurve(random_forest)
pr_curve.fit(X_train, y_train)
pr_curve.score(X_test, y_test)
pr_curve.show()


roc_curve = ROCAUC(random_forest, classes=["not_default", "default"])
roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
roc_curve.show() 


cm = ConfusionMatrix(random_forest, classes=['not default' , 'default'])
# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)
# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)
# How did we do?
cm.show()

In [ ]:
t1=time()
diff=np.array([t1-t0])

print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
times_rec=np.append(times_rec,diff)


# now let's get the accuracy and confusion matrix for this iterations of training/testing
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print ("classification report for iteration {} ".format(iter_num))

fig = plt.figure()
#ax = fig.add_subplot(211)

rf_cl_reporter = ClassificationReport(random_forest, classes=['non_default' , 'default'], support=True)
rf_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
rf_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
rf_cl_reporter.show()   

pr_curve = PrecisionRecallCurve(random_forest)
pr_curve.fit(X_train, y_train)
pr_curve.score(X_test, y_test)
pr_curve.show()


roc_curve = ROCAUC(random_forest, classes=["not_default", "default"])
roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
roc_curve.show() 


cm = ConfusionMatrix(random_forest, classes=['not default' , 'default'])
# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)
# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)
# How did we do?
cm.show()

In [ ]:
t0 = time()
random_forest = RandomForestClassifier(n_estimators=100, max_depth=5,
                         random_state=0, class_weight = 'balanced')

random_forest.fit(X_train,y_train)  # train object
y_hat = random_forest.predict(X_test) # get test set precitions

t1=time()
diff=np.array([t1-t0])

print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
times_rec=np.append(times_rec,diff)


# now let's get the accuracy and confusion matrix for this iterations of training/testing
acc = mt.accuracy_score(y_test,y_hat)
conf = mt.confusion_matrix(y_test,y_hat)
print ("classification report for iteration {} ".format(iter_num))

rf_cl_reporter = ClassificationReport(random_forest, classes=['non_default' , 'default'], support=True)

rf_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
rf_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
rf_cl_reporter.show()   

pr_curve = PrecisionRecallCurve(random_forest)
pr_curve.fit(X_train, y_train)
pr_curve.score(X_test, y_test)
pr_curve.show()


roc_curve = ROCAUC(random_forest, classes=["not_default", "default"])
roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
roc_curve.show() 


cm = ConfusionMatrix(random_forest, classes=['not default' , 'default'])
# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)
# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)
# How did we do?
cm.show()

###  Decision Tree classifiers 

We need to create some more  models by adjusting n_estimators and max_depth parameters. Also we need to  create one models with class_weight='balanced'

In [ ]:
Image(url ='../images/decision_tree_tan.png')

In [ ]:
iter_num=0
times_rec=[]
decision_tree_classifiers = dict()
random_forest_classifiers = dict()
max_acc_dt = 0
trn_idx_max_acc_dt =[]
tst_idx_max_acc_dt = []

for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
     
    decision_tree = DecisionTreeClassifier()

    decision_tree.fit(X_train,y_train)  # train object
    y_hat = decision_tree.predict(X_test) # get test set precitions
    
    t1=time()
    diff=np.array([t1-t0])
    
    print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
    times_rec=np.append(times_rec,diff)
    

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    if acc> max_acc_dt:
        max_acc_dt = acc
        trn_idx_max_acc_dt = train_indices
        tst_idx_max_acc_dt = test_indices
        
    conf = mt.confusion_matrix(y_test,y_hat)
    print ("classification report for iteration {} ".format(iter_num))
    
    
    dt_cl_reporter = ClassificationReport(decision_tree, classes=['non_default' , 'default'], support=True)

    dt_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
    dt_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
    dt_cl_reporter.show()  
    
    pr_curve = PrecisionRecallCurve(decision_tree)
    pr_curve.fit(X_train, y_train)
    pr_curve.score(X_test, y_test)
    pr_curve.show()
    
    roc_curve = ROCAUC(decision_tree, classes=["not_default", "default"])

    roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
    roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
    roc_curve.show() 
    
    cm = ConfusionMatrix(decision_tree, classes=['not default' , 'default'])
    # Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
    cm.fit(X_train, y_train)
    # To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
    # and then creates the confusion_matrix from scikit-learn.
    cm.score(X_test, y_test)
    # How did we do?
    cm.show()
    
        
    iter_num+=1
    

#### Decision tree with balanced class 
class weight impacts decison tree classifier as well and we 

In [ ]:
t0 = time()
     
decision_tree = DecisionTreeClassifier(class_weight = 'balanced')

decision_tree.fit(X_train,y_train)  # train object
y_hat = decision_tree.predict(X_test) # get test set precitions

t1=time()
diff=np.array([t1-t0])
print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
times_rec=np.append(times_rec,diff)
# now let's get the accuracy and confusion matrix for this iterations of training/testing
acc = mt.accuracy_score(y_test,y_hat)
print("accurancy for balanced class is {}".format(acc))
conf = mt.confusion_matrix(y_test,y_hat)
print ("classification report for iteration {} ".format(iter_num))
dt_cl_reporter = ClassificationReport(decision_tree, classes=['non_default' , 'default'], support=True)

dt_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
dt_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
dt_cl_reporter.show()  

pr_curve = PrecisionRecallCurve(decision_tree)
pr_curve.fit(X_train, y_train)
pr_curve.score(X_test, y_test)
pr_curve.show()

roc_curve = ROCAUC(decision_tree, classes=["not_default", "default"])

roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
roc_curve.show() 

cm = ConfusionMatrix(decision_tree, classes=['not default' , 'default'])
# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)
# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)
# How did we do?
cm.show()

iter_num+=1

### KNN 
TBD 

In [ ]:
parameters = [10, 20, 50, 100]
max_acc=0
for K in parameters:
    print("For k = ", K, "and metric = Euclidean: ")
    knn = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    knn.fit(X_train_scaled,y_train)
    y_hat = knn.predict(X_test_scaled)
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print('accuracy:', acc )
    print(conf)
    ClassReport = mt.classification_report(y_test,y_hat)
    print(ClassReport)
    max_acc = (acc) if acc>max_acc else (max_acc)
    
    print ("classification report for iteration {} ".format(iter_num))
    knn_cl_reporter = ClassificationReport(knn, classes=['non_default' , 'default'], support=True)

    knn_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
    knn_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
    knn_cl_reporter.show()  

    pr_curve = PrecisionRecallCurve(decision_tree)
    pr_curve.fit(X_train, y_train)
    pr_curve.score(X_test, y_test)
    pr_curve.show()

    roc_curve = ROCAUC(decision_tree, classes=["not_default", "default"])

    roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
    roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
    roc_curve.show() 

    cm = ConfusionMatrix(decision_tree, classes=['not default' , 'default'])
    # Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
    cm.fit(X_train, y_train)
    # To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
    # and then creates the confusion_matrix from scikit-learn.
    cm.score(X_test, y_test)
    # How did we do?
    cm.show()
    
#print(max_acc)    

## Modeling and Evaluation 4
We can give some details from the graphs created in previous section and explain those 

### Random Forest 
Random Forest gives 

### Decision Tree 

In [ ]:
trn_idx_max_acc_dt
tst_idx_max_acc_dt
X_train = X[trn_idx_max_acc_dt]
y_train = y[trn_idx_max_acc_dt]
X_test = X[tst_idx_max_acc_dt]
y_test = y[tst_idx_max_acc_dt]


decision_tree = DecisionTreeClassifier(class_weight = 'balanced')

decision_tree.fit(X_train,y_train)  # train object
y_hat = decision_tree.predict(X_test) # get test set precitions

t1=time()
diff=np.array([t1-t0])
print ("The time it takes to fit and predict for iteration {}  is {} ".format(iter_num, diff[0])) 
times_rec=np.append(times_rec,diff)
# now let's get the accuracy and confusion matrix for this iterations of training/testing
acc = mt.accuracy_score(y_test,y_hat)
print("accurancy for balanced class is {}".format(acc))
conf = mt.confusion_matrix(y_test,y_hat)
print ("classification report for iteration {} ".format(iter_num))
dt_cl_reporter = ClassificationReport(decision_tree, classes=['non_default' , 'default'], support=True)

dt_cl_reporter.fit(X_train, y_train)        # Fit the visualizer and the model
dt_cl_reporter.score(X_test, y_test)        # Evaluate the model on the test data
dt_cl_reporter.show()  

pr_curve = PrecisionRecallCurve(decision_tree)
pr_curve.fit(X_train, y_train)
pr_curve.score(X_test, y_test)
pr_curve.show()

roc_curve = ROCAUC(decision_tree, classes=["not_default", "default"])

roc_curve.fit(X_train, y_train)        # Fit the training data to the visualizer
roc_curve.score(X_test, y_test)        # Evaluate the model on the test data
roc_curve.show() 

cm = ConfusionMatrix(decision_tree, classes=['not default' , 'default'])
# Fit fits the passed model. This is unnecessary if you pass the visualizer a pre-fitted model
cm.fit(X_train, y_train)
# To create the ConfusionMatrix, we need some test data. Score runs predict() on the data
# and then creates the confusion_matrix from scikit-learn.
cm.score(X_test, y_test)
# How did we do?
cm.show()


As we can see from above 

### KNN 

## Modeling and Evaluation 5


### Random Forest 

In [ ]:

iter_num=0
times_rec=[]
decision_tree_classifiers = dict()
random_forest_classifiers = dict()

for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices] 
    X_test = X[test_indices]
    y_test = y[test_indices]
     #we count the time in executing the logistic regression
    t0 = time()
    random_forest = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
    viz = FeatureImportances(random_forest, labels = cc.columns)
    viz.fit(X_train_scaled,y_train)
    viz.show()

### Decision Tree 

In [ ]:
iter_num=0
times_rec=[]
decision_tree_classifiers = dict()
random_forest_classifiers = dict()

for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices] 
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
    random_forest = DecisionTreeClassifier()
    viz = FeatureImportances(decision_tree, labels = cc.columns)
    viz.fit(X_train_scaled,y_train)
    viz.show()

### KNN 

In case of knn there is no output for feature importance. We run into following error when we attempt to draw graph in same way in which we did for decision tree and random forest. 
In case of KNN we use particular metric to determine proximity of nodes and cluster them accordingly. The distnace is measured based on the difference between corresponding features.

In [ ]:
iter_num=0
times_rec=[]
 

for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices] 
    X_test = X[test_indices]
    y_test = y[test_indices]
    
     #we count the time in executing the logistic regression
    t0 = time()
    knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
    viz = FeatureImportances(knn, labels = cc.columns)
    viz.fit(X_train_scaled,y_train)
    viz.show()

## Modeling and Evaluation 6

Following is a Comparison of Decision Tree and Random forest classifiers with same date. We are using error estimates to compare two models. The error estimates should follow Gaussian distribution. We calculate mean and standard error using the formula discussed in the class. 
<TODO add image and formula >
    We are caclulating confidence interval for error differences to figure out if the differences are significant
  We can add similar test for knn , decision tree and knn and random forest comparison 


### Decision Tree and Random Forest Comparision based on error rate 

Our model comparison is based on following formula. 
We compare d bar by taking averedge  of the differences of the error rates of the models in different folds using same training set data.

In [ ]:
from IPython.display import Image
Image(url ='../images/model_comparison.png')

In [ ]:
iter_num=0
times_rec=[]
error_rec= [] 
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(X_train,y_train) # train object
    
    random_forest = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
    random_forest.fit(X_train,y_train) 
    
    y_hat_dt = decision_tree.predict(X_test) 
    y_hat_rf = random_forest.predict(X_test) 

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc_dt = mt.accuracy_score(y_test,y_hat_dt)
    acc_rf = mt.accuracy_score(y_test,y_hat_rf)
    error_rec.append(acc_dt - acc_rf)

    iter_num+=1

print(error_rec)

d_bar = sum(error_rec)/len(error_rec)
siqma_squared = sum((error_rec - d_bar))/(5-1)
print(d_bar)
print(siqma_squared)

from scipy import stats
import math

confidence_interval = ( d_bar + (1/math.sqrt(4))*stats.t.isf(.975, 4),  d_bar - (1/math.sqrt(4))*stats.t.isf(.975, 4))
print(confidence_interval)

As we can see above tht zero is a possible value for 95% confidence interval it means that we can't reject the null hypothesis that both the models have no difference in their error rate with 95% confidence.

##  KNN  and Random Forest Comparision based on error rate 

In [ ]:
iter_num=0
times_rec=[]
error_rec= [] 
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
    knn.fit(X_train,y_train) # train object
    
    random_forest = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
    random_forest.fit(X_train,y_train) 
    
    y_hat_knn = knn.predict(X_test) 
    y_hat_rf = random_forest.predict(X_test) 

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc_knn = mt.accuracy_score(y_test,y_hat_knn)
    acc_rf = mt.accuracy_score(y_test,y_hat_rf)
    error_rec.append(acc_knn - acc_rf)

    iter_num+=1

print(error_rec)

d_bar = sum(error_rec)/len(error_rec)
siqma_squared = sum((error_rec - d_bar))/(5-1)
print(d_bar)
print(siqma_squared)

from scipy import stats
import math

confidence_interval = ( d_bar + (1/math.sqrt(4))*stats.t.isf(.975, 4),  d_bar - (1/math.sqrt(4))*stats.t.isf(.975, 4))
print(confidence_interval)